In [1]:
#import data manipulation packages 
import pandas as pd 
import numpy as np 
import os
import cv2
import matplotlib
matplotlib.use('agg')
import matplotlib.pyplot as plt
import matplotlib.pyplot as plt
import warnings



In [2]:
#import deep learning tools 
from tensorflow.keras.layers import Input, Lambda, Dense, Flatten, Dropout, Concatenate
from tensorflow.keras.models import Model
from tensorflow.keras.preprocessing import image, image_dataset_from_directory
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.models import Sequential
from tensorflow import keras
import tensorflow

In [3]:
# Set the seed value for experiment reproducibility.
seed = 1842
tensorflow.random.set_seed(seed)
np.random.seed(seed)
# Turn off warnings for cleaner looking notebook
warnings.simplefilter('ignore')



In [5]:
#define image dataset 
#why do we rescale?
image_generator = ImageDataGenerator(rescale=1/255, validation_split=0.2) #shear_range =.25, zoom_range =.2, horizontal_flip = True, rotation_range=20)     

#Train & Validation Split 
train_dataset = image_generator.flow_from_directory(batch_size=10,
                                                 directory='data_cleaned/Train',
                                                 shuffle=True,
                                                 target_size=(224, 224), 
                                                 subset="training",
                                                 class_mode='categorical')

validation_dataset = image_generator.flow_from_directory(batch_size=10,
                                                 directory='data_cleaned/Train',
                                                 shuffle=True,
                                                 target_size=(224, 224), 
                                                 subset="validation",
                                                 class_mode='categorical')

#Organize data for our predictions 
image_generator_submission = ImageDataGenerator(rescale=1/255) 
submission = image_generator_submission.flow_from_directory(
                                                 directory='data_cleaned/scraped_images',
                                                 shuffle=False,
                                                 target_size=(224, 224), 
                                                 class_mode=None)


class_indices = train_dataset.class_indices
print(class_indices)


Found 430 images belonging to 2 classes.
Found 106 images belonging to 2 classes.
Found 100 images belonging to 1 classes.
{'la_eterna': 0, 'other_flowers': 1}


In [10]:
import matplotlib.pyplot as plt

# Get the first batch of data
batch_images, batch_labels = train_dataset.next()

# Define a function to convert one-hot encoded labels to class names
def decode_label(one_hot_label):
    return list(train_dataset.class_indices.keys())[np.argmax(one_hot_label)]

# Iterate over the batch
for i in range(len(batch_images)):
    image = batch_images[i]
    label = batch_labels[i]
    
    # Decode the label
    decoded_label = decode_label(label)
    
    # Display the image and label
    plt.imshow(image)
    plt.title(decoded_label)
    plt.show()



In [11]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense

# Define the layers
conv1 = Conv2D(15, 3, activation='relu', input_shape=[224, 224, 3])
maxpool1 = MaxPooling2D()
conv2 = Conv2D(15, 3, activation='relu')
maxpool2 = MaxPooling2D()
dropout = Dropout(0.5)
flatten = Flatten()
dense1 = Dense(64, activation='relu')
dense2 = Dense(2, activation='softmax')

# Construct the model
model = keras.models.Sequential([
    conv1,
    maxpool1,
    conv2,
    maxpool2,
    dropout,
    flatten,
    dense1,
    dense2
])


In [12]:
model.compile(loss = 'binary_crossentropy',optimizer='adam', metrics = ['accuracy'])

callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                         patience=3,
                                         restore_best_weights=True)

In [15]:
model.fit(train_dataset, epochs=5, validation_data=validation_dataset, callbacks=callback)
#save model
#early stopping

43/43 [==============================] - 4s 99ms/step - loss: 0.2132 - accuracy: 0.9140 - val_loss: 0.5308 - val_accuracy: 0.8019


In [16]:
loss, accuracy = model.evaluate(validation_dataset)
print("Loss:", loss)
print("Accuracy:", accuracy)

11/11 [==============================] - 1s 46ms/step - loss: 0.5308 - accuracy: 0.8019
Loss: 0.5307663679122925
Accuracy: 0.801886796951294


In [17]:
model.predict(submission);

4/4 [==============================] - 1s 110ms/step


In [18]:
model.summary()

Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d (Conv2D)             (None, 222, 222, 15)      420       
                                                                 
 max_pooling2d (MaxPooling2D  (None, 111, 111, 15)     0         
 )                                                               
                                                                 
 conv2d_1 (Conv2D)           (None, 109, 109, 15)      2040      
                                                                 
 max_pooling2d_1 (MaxPooling  (None, 54, 54, 15)       0         
 2D)                                                             
                                                                 
 dropout (Dropout)           (None, 54, 54, 15)        0         
                                                                 
 flatten (Flatten)           (None, 43740)             0

In [19]:
model.history

In [20]:
onlyfiles = [f.split('.')[0] for f in os.listdir(os.path.join('data_cleaned/scraped_images/image_files')) if os.path.isfile(os.path.join(os.path.join('data_cleaned/scraped_images/image_files'), f))]
submission_df = pd.DataFrame(onlyfiles, columns =['images'])
submission_df[['la_eterna','other_flower']] = model.predict(submission)
submission_df.head(100)

4/4 [==============================] - 0s 76ms/step


,images,la_eterna,other_flower
0,img_00,0.005924,0.994076
1,img_01,0.000040,0.999960
2,img_02,0.073693,0.926307
3,img_03,0.004001,0.995999
4,img_04,0.036987,0.963013
...,...,...,...
95,img_95,0.381451,0.618549
96,img_96,0.009613,0.990387
97,img_97,0.352000,0.648000
98,img_98,0.000417,0.999583


In [21]:
import os
import pandas as pd

# Assuming you've defined your model and loaded the images
# model = ...  # Your model definition
# submission = ...  # Your collection of images

image_files_dir = 'data_cleaned/scraped_images/image_files'
onlyfiles = [f.split('.')[0] for f in os.listdir(image_files_dir) if os.path.isfile(os.path.join(image_files_dir, f))]

submission_df = pd.DataFrame(onlyfiles, columns=['images'])

# Assuming your model predicts two classes and returns probabilities for each
# Change the prediction logic based on your model's output structure
predictions = model.predict(submission)
submission_df[['la_eterna', 'other_flower']] = predictions

submission_df.head()


4/4 [==============================] - 0s 71ms/step


,images,la_eterna,other_flower
0,img_00,0.005924,0.994076
1,img_01,0.000040,0.999960
2,img_02,0.073693,0.926307
3,img_03,0.004001,0.995999
4,img_04,0.036987,0.963013


In [23]:
import keras
from keras.layers import Conv2D, MaxPooling2D, Dropout, Flatten, Dense
from keras.preprocessing.image import ImageDataGenerator
# Construct the model
model_a = keras.models.Sequential([
    conv1,
    maxpool1,
    conv2,
    maxpool2,
    dropout,
    flatten,
    dense1,
    dense2
])

# Create an ImageDataGenerator instance with desired augmentation parameters
data_augmentation = ImageDataGenerator(
    rotation_range=10,
    width_shift_range=0.1,
    height_shift_range=0.1,
    zoom_range=0.1,
    horizontal_flip=True,
    brightness_range=[0.9, 1.1]
)



In [24]:
model_a.compile(loss = 'binary_crossentropy',optimizer='adam', metrics = ['accuracy'])

callback = keras.callbacks.EarlyStopping(monitor='val_loss',
                                         patience=3,
                                         restore_best_weights=True)

In [25]:
model_a.fit(train_dataset, epochs=5, validation_data=validation_dataset, callbacks=callback)

43/43 [==============================] - 5s 106ms/step - loss: 0.2162 - accuracy: 0.9070 - val_loss: 0.3128 - val_accuracy: 0.8396


In [26]:
loss, accuracy = model_a.evaluate(validation_dataset)
print("Loss:", loss)
print("Accuracy:", accuracy)

11/11 [==============================] - 0s 36ms/step - loss: 0.3128 - accuracy: 0.8396
Loss: 0.3127584159374237
Accuracy: 0.8396226167678833


In [27]:
x=model_a.predict(submission);


4/4 [==============================] - 0s 73ms/step


In [28]:
predicted_classes = np.argmax(x, axis=1)
print(predicted_classes)

[1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1
 1 1 1 1 0 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 0 1 1 1 1 1 1 1 0 1 1 1 1 1 1
 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1 1]


In [29]:
predicted_df = pd.DataFrame(predicted_classes, columns=['predicted_class'])

In [30]:
onlyfiles = [f.split('.')[0] for f in os.listdir(os.path.join('data_cleaned/scraped_images/image_files')) if os.path.isfile(os.path.join(os.path.join('data_cleaned/scraped_images/image_files'), f))]
submission_df = pd.DataFrame(onlyfiles, columns =['images'])
submission_df[['la_eterna','other_flower']] = model.predict(submission)
submission_df.head()

4/4 [==============================] - 0s 87ms/step


,images,la_eterna,other_flower
0,img_00,0.000018,0.999982
1,img_01,0.000008,0.999992
2,img_02,0.000373,0.999627
3,img_03,0.000010,0.999990
4,img_04,0.003462,0.996538


In [31]:
submission_df.to_csv('submission_file.csv', index = False)

# Assuming your model is named 'model'
model.save('model_a.h5')

